In [62]:
# Imports
import sys
import pygame

# Configuration
pygame.init()
fps = 60
fpsClock = pygame.time.Clock()
width, height = 1920, 1080
screen = pygame.display.set_mode((width, height))

font = pygame.font.SysFont('Arial', 12)

objects = []

class Button():
    def __init__(self, x, y, width, height, buttonText='\u9728', onclickFunction=None, onePress=False):
        self.x = x
        self.y = y
        self.offsetx = 0
        self.offsety = 0
        
        self.width = width
        self.height = height
        self.onclickFunction = onclickFunction
        self.onePress = onePress

        self.fillColors = {
            'normal': '#ffffff',
            'hover': '#666666',
            'pressed': '#333333',
        }
        self.text = buttonText
        objects.append(self)
        self.alreadyPressed = False
        self.time_down = pygame.time.get_ticks()

    def make_surface(self):

        self.buttonSurface = pygame.Surface((self.width, self.height))
        self.buttonRect = pygame.Rect(self.x, self.y, self.width, self.height)

        self.buttonSurf = font.render(self.text, True, (20, 20, 20))

        self.alreadyPressed = False

    def process(self):

        mousePos = pygame.mouse.get_pos()
        
        self.buttonSurface.fill(self.fillColors['normal'])
        if self.buttonRect.collidepoint(mousePos):
            self.buttonSurface.fill(self.fillColors['hover'])

            if pygame.mouse.get_pressed(num_buttons=3)[2]:
                self.buttonSurface.fill(self.fillColors['pressed'])

                if self.onePress:
                    self.onclickFunction()

                elif not self.alreadyPressed:
                    self.onclickFunction()
                    self.alreadyPressed = True

            else:
                self.alreadyPressed = False

        self.buttonSurface.blit(self.buttonSurf, [
            self.buttonRect.width/2 - self.buttonSurf.get_rect().width/2,
            self.buttonRect.height/2 - self.buttonSurf.get_rect().height/2
        ])
        screen.blit(self.buttonSurface, self.buttonRect)



class ContextMenu(Button):
    
    def __init__(self, x, y, buttonlist, onePress=False):

        self.buttons = buttonlist

        width = max(button.width for button in self.buttons)
        height = sum(button.height for button in self.buttons)

        Button.__init__(self, x, y, width, height )
        
        self.offsety = 50
        self.offsetx = 0

    def make_surface(self):
        self.x ,self.y = pygame.mouse.get_pos()
        self.x += self.offsetx
        self.y += self.offsety
        step = 0
        for button in self.buttons:
            button.x  = self.x
            button.y = self.y + step
            step+=button.height
            button.make_surface()


        Button.make_surface(self)
    
    def process(self):


        mousePos = pygame.mouse.get_pos()
        


        if not(self.alreadyPressed) and pygame.mouse.get_pressed(num_buttons=3)[2]:
            
            time_elapsed = (pygame.time.get_ticks() - self.time_down)/1000.0
            
            if time_elapsed > 0.1: 
                self.time_down = pygame.time.get_ticks()
                self.make_surface()
                self.alreadyPressed = True

        if self.alreadyPressed:
            for button in self.buttons:
                button.process()
            if not(self.buttonRect.collidepoint(mousePos)) and pygame.mouse.get_pressed(num_buttons=3)[0]:
                time_elapsed = (pygame.time.get_ticks() - self.time_down)/1000.0
                
                if time_elapsed > 0.1: 
                    self.alreadyPressed = False
                    self.time_down = pygame.time.get_ticks()

        
        
class MenuButton(Button):
    def __init__(self, x, y, width, height, buttonText='>', menu=None, onePress=False):
        self.menu = menu
        super().__init__(x, y, width, height, buttonText, self.menu.process, onePress)
        


        

    def process(self):
        mousePos = pygame.mouse.get_pos()
        self.buttonSurface.fill(self.fillColors['normal'])
        if self.buttonRect.collidepoint(mousePos):
            self.buttonSurface.fill(self.fillColors['hover'])

            if pygame.mouse.get_pressed(num_buttons=3)[2]:
                self.alreadyPressed = True
                self.buttonSurface.fill(self.fillColors['pressed'])

            elif pygame.mouse.get_pressed(num_buttons=3)[0]:
                self.alreadyPressed = False
        if self.alreadyPressed:
            self.menu.offsetx = -mousePos[0] + self.x + self.width
            self.menu.offsety =  -mousePos[1] + self.y 
            self.onclickFunction()

        self.buttonSurface.blit(self.buttonSurf, [
            self.buttonRect.width/2 - self.buttonSurf.get_rect().width/2,
            self.buttonRect.height/2 - self.buttonSurf.get_rect().height/2
        ])
        screen.blit(self.buttonSurface, self.buttonRect)

def myFunction():
    print('Button Pressed')



customButton = Button(0, 0, 200, 50, '\u221a', myFunction)
customButton = Button(0, 0, 400, 100, 'Button Two (multiPress)', myFunction, False)


customMenu = ContextMenu(0,0, objects[:])

objects = []

customButton = Button(0, 0, 400, 50, 'Button 3 (multiPress)', myFunction, False)
customButton = Button(0, 0, 400, 50, 'Button  3 (multiPress)', myFunction, False)

customButton2 = MenuButton(0, 0, 400, 50, '>', customMenu, False)



customMenu2 = ContextMenu(0,0, objects[:])


objects = []

customButton = Button(0, 0, 400, 50, 'Button 3 (multiPress)', myFunction, False)
customButton = Button(0, 0, 400, 50, 'Button  3 (multiPress)', myFunction, False)

customButton2 = MenuButton(0, 0, 400, 50, '>', customMenu2, False)



customMenu3 = ContextMenu(0,0, objects[:])

# Game loop.
while True:
    screen.fill((20, 20, 20))
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    # for object in objects:
    #     if object.visible :object.process()
    customMenu3.process()
    pygame.display.flip()
    fpsClock.tick(fps)

SystemExit: 